In [1]:
import langchain
from openai import OpenAI
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_community.llms import Ollama

In [2]:


#Needs PyPDF, langchain-community to load a PDF

#function for loading the documents to pass into the RAG model

def load_documents(data_path):
    loader = PyPDFLoader(data_path)
    document = loader.load_and_split()    #loader.load() returns a list of strings, each string is a page of the PDF. Use loader.load_and_split() instead
    return document

doc = load_documents("LLM MODEL SURVEY.pdf") #Docs can be retrieved by page numbers

# print(doc[0:2])

In [3]:
#Now, using text-splitting to get the text split from the PDF


def split_text(pdf_path):
    # 1. Load the PDF properly
    loader = PyPDFLoader(pdf_path)
    pages = loader.load()
    
    # 2. Create the text splitter
    text_splitter = RecursiveCharacterTextSplitter(
        #If you want bigger chunk size
        chunk_size=1000,
        #To overlap the context between chunks
        chunk_overlap=200,
        #Keep chunks above this size
        length_function=len,
        # You can add custom separators
        separators=["\n\n", "\n", " ", ""],
        is_separator_regex=True,
    )
    
    # 3. Split the documents
    chunks = text_splitter.split_documents(pages)
    print(f"Document split into {len(chunks)} chunks")
    return chunks


pdf_path = "LLM MODEL SURVEY.pdf"
chunks = split_text(pdf_path)

print(chunks[1])

Document split into 272 chunks
page_content='overview of techniques developed to build, and augment LLMs.
We then survey popular datasets prepared for LLM training,
fine-tuning, and evaluation, review widely used LLM evaluation
metrics, and compare the performance of several popular LLMs
on a set of representative benchmarks. Finally, we conclude
the paper by discussing open challenges and future research
directions.
I. I NTRODUCTION
Language modeling is a long-standing research topic, dat-
ing back to the 1950s with Shannon’s application of informa-
tion theory to human language, where he measured how well
simple n-gram language models predict or compress natural
language text [3]. Since then, statistical language modeling
became fundamental to many natural language understanding
and generation tasks, ranging from speech recognition, ma-
chine translation, to information retrieval [4], [5], [6].
The recent advances on transformer-based large language
models (LLMs), pretrained on Web-s

In [4]:
#Now, we have to add embedding for each of the chunks from the document
from langchain_huggingface import HuggingFaceEmbeddings
''' when creating the embedding function, make sure to use the same embedding when creating the database 
    and also when we want to query the database'''

def get_embedding_function():
    embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [5]:
#Fetching the embedding function
embeddings = get_embedding_function()

c:\Users\Asus\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [14]:
#Creating the database with FAISS
faiss_index = FAISS.from_documents(chunks, embeddings)

#Performing a similarity search
# Function to retrieve top 3 relevant documents
def retrieve_documents(query, top_k=3):
    results = faiss_index.similarity_search(query, k=top_k)
    return [doc.page_content for doc in results]

# Test the system
query = "WGPT-4 performance compared to other models"
retrieved_docs = retrieve_documents(query)

# Print the retrieved documents
print("\nTop relevant documents:")
for idx, doc in enumerate(retrieved_docs, 1):
    print(f"{idx}. {doc}")


combined_text = "\n\n".join(retrieved_docs)


Top relevant documents:
1. outperforming other models like LLaMA and Stanford Alpaca
in more than 90% of cases. 13 shows the relative response
quality of Vicuna and a few other well-known models by
GPT-4. Another advantage of Vicuna-13B is its relative limited
computational demand for model training. The training cost of
Vicuna-13B is merely $300.
Fig. 13: Relative Response Quality of Vicuna and a few other
well-known models by GPT-4. Courtesy of Vicuna Team.
Like Alpaca and Vicuna, the Guanaco models [63] are also
finetuned LLaMA models using instruction-following data. But
the finetuning is done very efficiently using QLoRA such
that finetuning a 65B parameter model can be done on a
single 48GB GPU. QLoRA back-propagates gradients through
a frozen, 4-bit quantized pre-trained language model into Low
Rank Adapters (LoRA). The best Guanaco model outperforms
all previously released models on the Vicuna benchmark,
reaching 99.3% of the performance level of ChatGPT while
2. LLAMA 2 70B -

In [ ]:
#Now, creating the LLM model

llm = OpenAI(api_key = 'INSERT_GOOGLE_API_KEY_HERE', 
             base_url="https://generativelanguage.googleapis.com/v1beta/openai/")


prompt = f"give summary of the following text, only that and no extra text: {combined_text}"
messages = [{'role' : 'system', 'content' : 'You are a helpful assistant that gives concise information.'},
            {'role' : 'user', 'content' : prompt}]

response = llm.chat.completions.create(model = 'gemini-2.5-flash-preview-04-17', reasoning_effort="low",
                                        messages = messages)

print(response.choices[0].message.content)

Vicuna-13B is a cost-efficient model ($300 training) outperforming LLaMA and Alpaca in most cases, with its quality assessed by GPT-4. Guanaco models, efficiently finetuned using QLoRA, achieve high performance (99.3% of ChatGPT on Vicuna benchmark). Benchmark results are presented for various models on tasks like HellaSwag and OBQA, showing varying performance with GPT-4 and Davinci-003 often leading in specific areas. Fine-tuning, even for strong models like GPT-4, can further improve performance on specific tasks.
